# 02. Predict Employee Attrition


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Install all related packages. If you find additional packages missing, please follow the same technique.
#If you are not using anaconda, then use pip to install the same packages

import sys
!conda install --yes --prefix {sys.prefix} pandas tensorflow

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/hr

  added / updated specs:
    - pandas
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-1.0.4               |   py37h959d312_0         7.9 MB
    ------------------------------------------------------------
                                           Total:         7.9 MB

The following NEW packages will be INSTALLED:

  _tflow_select      pkgs/main/osx-64::_tflow_select-2.3.0-mkl
  absl-py            pkg

## 02.04. Preprocessing Attrition Data

In [2]:
#Load the dataset and analyze

import pandas as pd
import os
import tensorflow as tf
import numpy as np

attrition_data = pd.read_csv("/content/drive/MyDrive/Ai in HR/employee_attrition.csv")

print("Data Loaded:\n------------------------\n",attrition_data.dtypes)
attrition_data.head()

Data Loaded:
------------------------
 EmployeeID              int64
TotalMonthsOfExp        int64
TotalOrgsWorked         int64
MonthsInOrg             int64
LastPayIncrementBand    int64
AverageFeedback         int64
LastPromotionYears      int64
Attrition               int64
dtype: object


,EmployeeID,TotalMonthsOfExp,TotalOrgsWorked,MonthsInOrg,LastPayIncrementBand,AverageFeedback,LastPromotionYears,Attrition
0,1,110,4,9,5,4,4,1
1,2,103,3,51,1,4,2,0
2,3,41,4,16,5,4,4,1
3,4,32,4,17,5,2,3,0
4,5,80,3,16,3,4,2,0


In [3]:
#Correlation Analysis of target attribute

attrition_data.corr()['Attrition']

EmployeeID             -0.036630
TotalMonthsOfExp        0.019702
TotalOrgsWorked         0.008706
MonthsInOrg             0.012605
LastPayIncrementBand    0.108528
AverageFeedback        -0.008253
LastPromotionYears      0.765641
Attrition               1.000000
Name: Attrition, dtype: float64

In [4]:
#Convert to Dataframe to numpy array
np_attrition = attrition_data.to_numpy().astype(float)

#Create X_train with the first 7 attributes
X_train = np_attrition[:,1:7]
#Create Y_train with attrition attribute
Y_train=np_attrition[:,7]

#Convert Y_train to one-hot-encoding
Y_train = tf.keras.utils.to_categorical(Y_train,2)

print("X-Train Shape : ", X_train.shape)
print("Y-Train Shape : ", Y_train.shape)

X-Train Shape :  (1000, 6)
Y-Train Shape :  (1000, 2)


## 02.05. Build Attrition model with Keras

In [5]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Setup hyperparameters for deep learning
EPOCHS=100
BATCH_SIZE=100
VERBOSE=1
NB_CLASSES=2
N_HIDDEN=128
VALIDATION_SPLIT=0.2

#Create a Keras model
model = tf.keras.models.Sequential()

#Add first hidden Dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(6,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second hidden dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a final layer with softmax
model.add(keras.layers.Dense(NB_CLASSES,
                             name='Final',
                             activation='softmax'))

#Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Fit parameters 
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

Epoch 1/100
8/8 [==============================] - 1s 29ms/step - loss: 1.8922 - accuracy: 0.6162 - val_loss: 1.7289 - val_accuracy: 0.7850
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 1.2510 - accuracy: 0.7088 - val_loss: 1.1717 - val_accuracy: 0.2750
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.8016 - accuracy: 0.6562 - val_loss: 0.6613 - val_accuracy: 0.7850
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5840 - accuracy: 0.7638 - val_loss: 0.5624 - val_accuracy: 0.7750
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5138 - accuracy: 0.7925 - val_loss: 0.5140 - val_accuracy: 0.7700
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4788 - accuracy: 0.7850 - val_loss: 0.4883 - val_accuracy: 0.7750
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4631 - accuracy: 0.7950 - val_loss: 0.4695 - val_accuracy: 0.7800
Epoch 8/100
8/8 [=========

## 02.06. Predict Attrition with Keras

In [6]:
TotalMonthsOfExp=40
TotalOrgsWorked=4
MonthsInOrg=20
LastPayIncrementBand=5
AverageFeedback=4
LastPromotionYears=4

print("Will employee leave ?", model.predict_classes([[TotalMonthsOfExp,
                                  TotalOrgsWorked,
                                  MonthsInOrg,
                                  LastPayIncrementBand,
                                  AverageFeedback,
                                  LastPromotionYears]]))

Will employee leave ? [1]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [7]:
#Bulk predictions

print(model.predict_classes(
    [[111,5,85,3,2,2],
    [31,2,15,4,1,4],
    [61,4,24,1,4,3],
    [77,4,35,3,1,1],
    [81,5,7,1,2,3],
    [113,4,112,5,4,1],
    [101,2,48,5,1,4],
    [45,4,22,5,3,1],
    [25,2,2,2,3,2],
    [97,3,15,3,2,4]]))

[0 1 0 0 0 0 1 0 0 1]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
